## LSA 실습

### 1. 환경설정

In [1]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 34.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.datasets import fetch_20newsgroups
from gensim import corpora, matutils
from gensim.models import TfidfModel
from gensim.models import lsimodel
import matplotlib.pyplot as plt
from pprint import pprint

# NLTK 데이터 다운로드
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

### 2. 코퍼스 정의

In [3]:
# 데이터셋 로드
categories = ['alt.atheism', 'talk.religion.misc',
              'comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(
    subset='train',
    categories=categories,
    remove=('headers', 'footers', 'quotes'),
    random_state=123
)

documents = newsgroups_train.data

print(f"전체 문서 수: {len(documents)}")

전체 문서 수: 2034


In [4]:
# 불용어 및 lemmatizer 설정
stopwords_en = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# 텍스트 전처리 함수
def text_to_tokens(text):
    text = text.lower()  # 소문자 변환
    tokens = nltk.word_tokenize(text)  # 토큰화
    tokens = [lemmatizer.lemmatize(t) for t in tokens]  # 표제어 추출
    tokens = [t for t in tokens if t not in stopwords_en]  # 불용어 제거
    tokens = [t for t in tokens if re.fullmatch('[a-z]+', t)]  # 알파벳만 유지
    return tokens

# 문서 전처리
doc_tokenized = [text_to_tokens(doc) for doc in documents]
print(f"첫 번째 문서의 토큰 수: {len(doc_tokenized[0])}")
print(f"토큰 샘플: {doc_tokenized[0][:10]}")

# Gensim 사전 및 BoW 생성
dictionary = corpora.Dictionary(doc_tokenized)
bow_corpus = [dictionary.doc2bow(doc) for doc in doc_tokenized]

print(f"단어 수: {len(dictionary)}")
print(f"문서 수: {len(bow_corpus)}")

첫 번째 문서의 토큰 수: 8
토큰 샘플: ['file', 'format', 'autodesk', 'animation', 'studio', 'available', 'thanks', 'gary']
단어 수: 19886
문서 수: 2034


### 3. LSA 모델 적용 - TF-IDF 행렬
* 코퍼스를 TF-IDF 행렬을 기반으로 셍성

In [5]:
tfidf_model = TfidfModel(bow_corpus, smartirs='nfc')
tfidf_corpus = tfidf_model[bow_corpus]

In [6]:
# 세 번째 문서에서 가장 빈도가 높은 단어 찾기
doc2_bow = bow_corpus[2]
most_frequent = max(doc2_bow, key=lambda x: x[1])  # (단어ID, 빈도) 중 빈도가 최대인 것
word_id, frequency = most_frequent
word = dictionary[word_id]

print(f"세 번째 문서에서 가장 많이 등장하는 단어: '{word}' ({frequency}번)")

세 번째 문서에서 가장 많이 등장하는 단어: 'paintbrush' (3번)


In [7]:
tfidf_matrix = matutils.corpus2dense(tfidf_corpus, num_terms=len(dictionary)).transpose()
print(f"해당 단어의 인덱스: {word_id}")
print(f"해당 단어의 TF-IDF 값: {tfidf_matrix[2, word_id]:.4f}")

해당 단어의 인덱스: 71
해당 단어의 TF-IDF 값: 0.6659


* TF-IDF 행렬을 토픽 수가 4개인 LSA 모델에 적용

In [8]:
# LSA(LSI) 모델 적용
lsi_model = lsimodel.LsiModel(corpus=tfidf_corpus, id2word=dictionary, num_topics=4, random_seed=123)

### 4. 토픽별 단어 할당값

In [9]:
# 토픽별 단어 할당값 행렬
topic_word_assignment_mat = lsi_model.get_topics()
print("토픽별 단어 할당값의 행렬 표현:")
pprint(np.round(topic_word_assignment_mat, 3))

print("토픽별 중요한 단어 순서대로 표현:")
pprint(lsi_model.print_topics())

토픽별 단어 할당값의 행렬 표현:
array([[ 0.019,  0.002,  0.044, ...,  0.   ,  0.   ,  0.   ],
       [ 0.046,  0.009,  0.082, ...,  0.   ,  0.   ,  0.   ],
       [ 0.026,  0.009,  0.003, ...,  0.   ,  0.   ,  0.   ],
       [ 0.001,  0.   , -0.016, ...,  0.   ,  0.   ,  0.   ]])
토픽별 중요한 단어 순서대로 표현:
[(0,
  '0.153*"wa" + 0.150*"god" + 0.140*"would" + 0.120*"one" + 0.119*"people" + '
  '0.108*"doe" + 0.107*"think" + 0.104*"space" + 0.103*"know" + 0.096*"like"'),
 (1,
  '-0.275*"god" + 0.256*"file" + 0.191*"image" + 0.161*"thanks" + '
  '0.152*"format" + 0.151*"program" + -0.142*"jesus" + 0.134*"graphic" + '
  '-0.127*"christian" + 0.111*"color"'),
 (2,
  '-0.341*"space" + 0.238*"file" + 0.204*"god" + -0.164*"launch" + '
  '-0.149*"nasa" + 0.136*"thanks" + 0.126*"format" + -0.126*"orbit" + '
  '-0.119*"satellite" + -0.110*"shuttle"'),
 (3,
  '-0.294*"beauchaine" + -0.294*"bronx" + -0.292*"manhattan" + -0.291*"bobbe" '
  '+ -0.290*"sank" + -0.281*"blew" + -0.279*"queen" + -0.267*"bob" + '
  '-0.263*"se

In [12]:
print(np.abs(topic_word_assignment_mat[2,:]))
# 가장 큰 값의 인덱스 도출
print("가장 큰 값을 가진 인덱스:",np.argmax(np.abs(topic_word_assignment_mat[2,:])))

[0.02627535 0.00897303 0.00337168 ... 0.00010914 0.00010914 0.00010914]
가장 큰 값을 가진 인덱스: 119


In [13]:
print("세번째 토픽의 가장 중요한 단어:", np.argmax(np.abs(topic_word_assignment_mat[2,:])))
print("인덱스 119의 단어는", dictionary[119])
print("세번째 토픽의 인덱스 119 단어의 할당값은", topic_word_assignment_mat[2,119])

세번째 토픽의 가장 중요한 단어: 119
인덱스 119의 단어는 space
세번째 토픽의 인덱스 119 단어의 할당값은 -0.3411927883878691


### 5. 문서별 토픽 할당값

In [19]:
# 문서별 토픽 할당값
doc_topic_assignment = lsi_model[bow_corpus]
print("문서별 토픽 할당값의 간단한 표현:")
pprint(list(doc_topic_assignment))

# 문서별 토픽 할당값 행렬 표현
doc_topic_assignment_mat = np.zeros(shape=(len(doc_topic_assignment), lsi_model.num_topics), dtype=float)
for doc_idx, doc_topics in enumerate(doc_topic_assignment):
    for topic_idx, topic_weight in doc_topics:
        doc_topic_assignment_mat[doc_idx, topic_idx] = topic_weight

print("문서별 토픽 할당값의 행렬 표현:")
pprint(np.round(doc_topic_assignment_mat, 3))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 [(np.int64(0), np.float64(2.8582795174388047)),
  (np.int64(1), np.float64(-1.0518153372973513)),
  (np.int64(2), np.float64(0.27960775828874185)),
  (np.int64(3), np.float64(0.001135479922183582))],
 [(np.int64(0), np.float64(2.5354472492654563)),
  (np.int64(1), np.float64(0.35256244835200645)),
  (np.int64(2), np.float64(0.42438234424570725)),
  (np.int64(3), np.float64(0.4525917455757814))],
 [(np.int64(0), np.float64(2.057858408796304)),
  (np.int64(1), np.float64(-0.6260355267889474)),
  (np.int64(2), np.float64(0.47285001000322024)),
  (np.int64(3), np.float64(-0.01589024298779684))],
 [(np.int64(0), np.float64(0.9075665508284179)),
  (np.int64(1), np.float64(0.18201318408127526)),
  (np.int64(2), np.float64(-0.4041575984097201)),
  (np.int64(3), np.float64(-0.03606145281573105))],
 [(np.int64(0), np.float64(0.5297642672350397)),
  (np.int64(1), np.float64(0.20558993794189134)),
  (np.int64(2), np.float64(0.20572204129067354)),
  (np.int64(3)

In [20]:
print("첫번째 문서의 토픽 할당값", doc_topic_assignment_mat[0,:])

첫번째 문서의 토픽 할당값 [ 0.28331719  0.73270847  0.55167789 -0.00915064]


* 절댓값이 두번째 토픽이 가장 크므로 두번째 토픽의 영향력이 큼

### 6. 토픽별 코퍼스 내 중요도 정보

In [21]:
singular_values = lsi_model.projection.s
print("토픽별 코퍼스 내 중요도(특이값):", singular_values)

토픽별 코퍼스 내 중요도(특이값): [5.41822103 3.50597295 2.9812098  2.69143663]


In [22]:
print("첫 번째 토픽의 특이값:", singular_values[0])

첫 번째 토픽의 특이값: 5.418221033742447


* 첫번째 토픽이 코퍼스 내에서 가장 중요(첫번째 토픽이 문서들의 주요 패턴을 가장 잘 포착한다고 볼 수 있음)